# **Projeto IA - Connect 4**

### Membros do Grupo:
- Maximiliano Vítor Phillips e Sá (up202305979)
- Orlando Miguel Carvalho-Soares (up202303606)
- Rui Jorge Pereira Rua (up202305259)

## Índice
1. Introdução
2. Código Universal
3. Game Modes
    - Humano x Humano
    - Humano x Computador
    - Computador x Computador


## 1. Introdução

Este projeto tem como objetivo desenvolver e implementar um programa capaz de jogar 'Connect-4' com um humano. O programa tem 3 modos: Humano-Humano, Humano-Computador e Computador-Computador. O programa utiliza um algoritmo MCTS no modo Humano-Computador e os algoritmos MCTS e Árvores de Decisão usando o procedimento ID3 no modo Computador-Computador.


## 2. Código Universal

meta.py

In [ ]:
import math

#meta dados relativos ao jogo
class GameMeta:
    PLAYERS = {'none': 0, 'one': 1, 'two': 2}
    OUTCOMES = {'none': 0, 'one': 1, 'two': 2, 'draw': 3}
    INF = float('inf')
    ROWS = 6
    COLS = 7
    GAMEMODE = None

#valor C do UCB
class MCTSMeta:
    C = math.sqrt(2)


ConnectState.py

In [ ]:
from copy import deepcopy
import numpy as np
from meta import GameMeta


class ConnectState:
    '''
        Construtor do board e de alguns metadados sobre o estado atual. 
        Ex: jogador atual, jogada anterior, altura de cada coluna (height[col] = 0 -> coluna col cheia)
    '''

    def __init__(self):
        self.board = [[0] * GameMeta.COLS for _ in range(GameMeta.ROWS)]
        self.to_play = GameMeta.PLAYERS['one']
        self.height = [GameMeta.ROWS - 1] * GameMeta.COLS
        self.last_played = []

    #retorna uma cópia do estado atual
    def get_board(self):
        return deepcopy(self.board)
    
    #dá print_board ao board atual na consola
    def print_board(self):
        print('=============================')

        for row in range(GameMeta.ROWS):
            for col in range(GameMeta.COLS):
                print('| {} '.format('X' if self.board[row][col] == 1 else 'O' if self.board[row][col] == 2 else ' '), end='')
            print('|')

        print('=============================')
        print('  1   2   3   4   5   6   7')

    #marca a jogada escolhida pelo jogador no board, e dá update a alguns metadados do estado atual
    def move(self, col):
        self.board[self.height[col]][col] = self.to_play
        self.last_played = [self.height[col], col]
        self.height[col] -= 1
        self.to_play = GameMeta.PLAYERS['two'] if self.to_play == GameMeta.PLAYERS['one'] else GameMeta.PLAYERS['one']

    #retorna uma lista com as jogadas que ainda não estão cheias (height[col] != 0)
    def get_legal_moves(self):
        return [col for col in range(GameMeta.COLS) if self.board[0][col] == 0]

    #verfica se há vencedor e retorna o seu número (X = 1 ; O = 2), se não houver retorna 0
    def check_win(self):
        if len(self.last_played) > 0 and self.check_win_from(self.last_played[0], self.last_played[1]):
            return self.board[self.last_played[0]][self.last_played[1]]
        return 0

    def check_win_from(self, row, col):
        player = self.board[row][col]
        """
        Ultima jogada realizada está na posição (row, col)
        Verificar a grid 6x7 à volta à procura de um vencedor
        """

        consecutive = 1
        # verificar horizontal
        tmprow = row
        while tmprow + 1 < GameMeta.ROWS and self.board[tmprow + 1][col] == player:
            consecutive += 1
            tmprow += 1
        tmprow = row
        while tmprow - 1 >= 0 and self.board[tmprow - 1][col] == player:
            consecutive += 1
            tmprow -= 1

        if consecutive >= 4:
            return True

        # verificar vertical
        consecutive = 1
        tmpcol = col
        while tmpcol + 1 < GameMeta.COLS and self.board[row][tmpcol + 1] == player:
            consecutive += 1
            tmpcol += 1
        tmpcol = col
        while tmpcol - 1 >= 0 and self.board[row][tmpcol - 1] == player:
            consecutive += 1
            tmpcol -= 1

        if consecutive >= 4:
            return True

        # verificar diagonal1
        consecutive = 1
        tmprow = row
        tmpcol = col
        while tmprow + 1 < GameMeta.ROWS and tmpcol + 1 < GameMeta.COLS and self.board[tmprow + 1][tmpcol + 1] == player:
            consecutive += 1
            tmprow += 1
            tmpcol += 1
        tmprow = row
        tmpcol = col
        while tmprow - 1 >= 0 and tmpcol - 1 >= 0 and self.board[tmprow - 1][tmpcol - 1] == player:
            consecutive += 1
            tmprow -= 1
            tmpcol -= 1

        if consecutive >= 4:
            return True

        # verificar diagonal2
        consecutive = 1
        tmprow = row
        tmpcol = col
        while tmprow + 1 < GameMeta.ROWS and tmpcol - 1 >= 0 and self.board[tmprow + 1][tmpcol - 1] == player:
            consecutive += 1
            tmprow += 1
            tmpcol -= 1
        tmprow = row
        tmpcol = col
        while tmprow - 1 >= 0 and tmpcol + 1 < GameMeta.COLS and self.board[tmprow - 1][tmpcol + 1] == player:
            consecutive += 1
            tmprow -= 1
            tmpcol += 1

        if consecutive >= 4:
            return True

        return False

    #verifica se o jogo já acabou verificando se há vencedor ou se ainda existem jogadas possíveis
    def game_over(self):
        return self.check_win() or len(self.get_legal_moves()) == 0

    #verifica quem ganhou ou se é empate
    def get_result(self):
        if len(self.get_legal_moves()) == 0 and self.check_win() == 0:
            return GameMeta.OUTCOMES['draw']

        return GameMeta.OUTCOMES['one'] if self.check_win() == GameMeta.PLAYERS['one'] else GameMeta.OUTCOMES['two']



## 3. Game Modes